In [71]:
import pandas as pd

# Read json file
df = pd.read_json('../../data/processed/topic_data.json')
df.head()

,_id,created_at_dt,reply_count,retweet_count,favourite_count,quote_count,sentiment,main_topic,fake_news_pred
0,1192617057394708480,1573177038000,0,280,458,0,Positive,1,0
1,1192684541913100288,1573193128000,0,609,673,0,Neutral,1,0
2,1192725213294804992,1573202825000,0,118,158,0,Positive,8,0
3,1192766676854435840,1573212710000,0,2879,5002,0,Negative,1,0
4,1192932983407202304,1573252361000,26,364,639,85,Neutral,1,0


In [60]:
import bson

# Read bson file
with open('../../data/raw/v_forest.bson', 'rb') as file:
    data = bson.decode_all(file.read())

ef = pd.DataFrame(data)
ef.head()

,_id,author,parent,quoted,mentions,hashtags,depth,node_type,quote_ancestors,tweet_ancestors,...,deidentmentions,deidentscreenname,deidentname,timebucket,horiz_offset,influence_tweet_factor,left,vert_offset,location,vert_correct
0,3787575861,34116377,None,None,[34116377],[],0,Root,[],[],...,"[[Scott Morrison, ScottMorrisonMP]]",ScottMorrisonMP,Scott Morrison,2009-09-05 23:00,0.5,1.767156,True,0.437778,"Australian Capital Territory, Australia",True
1,5433639942,34116377,None,None,[34116377],[],0,Root,[],[],...,"[[Scott Morrison, ScottMorrisonMP]]",ScottMorrisonMP,Scott Morrison,2009-11-04 23:15,0.5,0.778151,False,0.320000,"Australian Capital Territory, Australia",True
2,289077574649409537,335469402,None,None,[335469402],"[bushfires, scorchio, australia, weather]",0,Root,[],[],...,"[[Hugh Cook-Bullock, HughCookBullock]]",HughCookBullock,Hugh Cook-Bullock,2013-01-09 18:30,0.5,0.602060,True,0.265556,United Kingdom,True
3,316243081693384705,335469402,None,None,"[347665801, 335469402, 288009920, 280854233, 3...","[neighboursuk, easterbilby]",0,Root,[],[],...,"[[Rosa-Maria Stokes, RosaMariaStokes], [Lesley...",RoselMarieJohn,Rosel Marie John,2013-03-25 17:30,0.5,0.845098,False,0.666667,United Kingdom,True
4,345123067816706048,1512911756,None,None,[1512911756],[],0,Root,[],[],...,"[[Billy Butler, BillyButler]]",BillyButler,Billy Butler,2013-06-13 10:15,0.5,1.176091,True,0.251111,Unknown,True


In [72]:
df['_id'] = df['_id'].astype(str)

In [73]:
ef['_id'] = ef['_id'].astype(str)

In [74]:
# Merging two DataFrames 'df' and 'ef' on the '_id' column, with suffixes added to overlapping columns from 'ef'
merged_df = pd.merge(df, ef, on='_id', suffixes=('', '_ef'))
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157670 entries, 0 to 157669
Data columns (total 51 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   _id                     157670 non-null  object        
 1   created_at_dt           157670 non-null  int64         
 2   reply_count             157670 non-null  int64         
 3   retweet_count           157670 non-null  int64         
 4   favourite_count         157670 non-null  int64         
 5   quote_count             157670 non-null  int64         
 6   sentiment               157670 non-null  object        
 7   main_topic              157670 non-null  int64         
 8   fake_news_pred          157670 non-null  int64         
 9   author                  157670 non-null  object        
 10  parent                  0 non-null       object        
 11  quoted                  41600 non-null   object        
 12  mentions                157670

In [84]:
merged_df = merged_df.rename(columns={'main_topic': 'dominant_topic'})

In [3]:
merged_df.info()

NameError: name 'merged_df' is not defined

In [86]:
# Counts the number of media items in 'extended_entities'
def count_media(extended_entities):
    if isinstance(extended_entities, dict) and 'media' in extended_entities:
        return len(extended_entities['media'])
    return 0

merged_df['extended_entities_count'] = merged_df['extended_entities'].apply(count_media)

In [87]:
merged_df['sentiment'] = merged_df['sentiment'].apply(lambda x: x.capitalize() if isinstance(x, str) else 'Neutral')

In [88]:
df_simplified = merged_df[["_id", "deidentname", "text", "created_at_dt", "location", "dominant_topic",
                   "sentiment", "influence_tweet_factor", "influence_user", "retweet_count",
                   "reply_count", "quote_count", "favourite_count", "node_type",
                   "author_keynode", "hashtag_keynode", "extended_entities_count", "verified"]]
df_simplified.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157670 entries, 0 to 157669
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   _id                      157670 non-null  object 
 1   deidentname              157670 non-null  object 
 2   text                     157670 non-null  object 
 3   created_at_dt            157670 non-null  int64  
 4   location                 157670 non-null  object 
 5   dominant_topic           157670 non-null  int64  
 6   sentiment                157670 non-null  object 
 7   influence_tweet_factor   157670 non-null  float64
 8   influence_user           157670 non-null  float64
 9   retweet_count            157670 non-null  int64  
 10  reply_count              157670 non-null  int64  
 11  quote_count              157670 non-null  int64  
 12  favourite_count          157670 non-null  int64  
 13  node_type                157670 non-null  object 
 14  auth

In [90]:
df_simplified.head()

,_id,deidentname,text,created_at_dt,location,dominant_topic,sentiment,influence_tweet_factor,influence_user,retweet_count,reply_count,quote_count,favourite_count,node_type,author_keynode,hashtag_keynode,extended_entities_count,verified
0,1192617057394708480,Armin Mudden,If that’s what the fires look like from up her...,1573177038000,"New South Wales, Australia",1,Positive,2.448706,541.0,280,0,0,458,Root,False,False,1,False
1,1192684541913100288,Linda Roux,Progression of major fires today across NSW #N...,1573193128000,"New South Wales, Australia",1,Neutral,2.974742,12942.0,609,0,0,673,Root,False,True,1,True
2,1192725213294804992,Carolyn Butler,#Smoke from #nswfires in the northeast is not ...,1573202825000,"New South Wales, Australia",8,Positive,2.343409,148708.5,118,0,0,158,Root,False,True,2,True
3,1192766676854435840,NSW RFS,This shows the dangerous conditions that have ...,1573212710000,"New South Wales, Australia",1,Negative,3.635333,297835.5,2879,0,0,5002,Root,True,True,1,True
4,1192932983407202304,Stephen Gösta Marquez-Lindkvist,Incredible vision from @NSWRFS of a crew from ...,1573252361000,Australia,1,Neutral,2.829304,19101.0,364,26,85,639,Root,False,True,1,True


In [91]:
df_simplified["influence_tweet_factor"] = df_simplified["influence_tweet_factor"].round(2)

/var/folders/sn/vtf5wp8x6vldjnk3w88mk9zr0000gn/T/ipykernel_4857/3819418713.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_simplified["influence_tweet_factor"] = df_simplified["influence_tweet_factor"].round(2)


In [92]:
from datetime import datetime
# Converts timestamp (in milliseconds) to ISO 8601 format
def convert_timestamp(ts):
    return datetime.fromtimestamp(int(ts) / 1000).isoformat()

# Applies the 'convert_timestamp' function to the 'created_at_dt' column in 'df_simplified'
df_simplified['created_at_dt'] = df_simplified['created_at_dt'].apply(convert_timestamp)

/var/folders/sn/vtf5wp8x6vldjnk3w88mk9zr0000gn/T/ipykernel_4857/2512802749.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_simplified['created_at_dt'] = df_simplified['created_at_dt'].apply(convert_timestamp)


In [2]:
df_simplified.head()

NameError: name 'df_simplified' is not defined

In [1]:
df_simplified.info()

NameError: name 'df_simplified' is not defined

In [95]:
df_simplified.to_csv('../../data/processed/Search_data.csv')

In [4]:
import pandas as pd
df = pd.read_csv('../../data/processed/Search_data.csv') 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157670 entries, 0 to 157669
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Unnamed: 0.1             157670 non-null  int64  
 1   Unnamed: 0               157670 non-null  int64  
 2   _id                      157670 non-null  int64  
 3   deidentname              157670 non-null  object 
 4   text                     157670 non-null  object 
 5   created_at_dt            157670 non-null  object 
 6   location                 157670 non-null  object 
 7   dominant_topic           157670 non-null  int64  
 8   sentiment                157670 non-null  object 
 9   influence_tweet_factor   157670 non-null  float64
 10  influence_user           157670 non-null  float64
 11  retweet_count            157670 non-null  int64  
 12  reply_count              157670 non-null  int64  
 13  quote_count              157670 non-null  int64  
 14  favo

In [7]:
# Drops the columns 'Unnamed: 0.1' and 'Unnamed: 0' from the DataFrame 'df'
df = df.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)
df.head()

,_id,deidentname,text,created_at_dt,location,dominant_topic,sentiment,influence_tweet_factor,influence_user,retweet_count,reply_count,quote_count,favourite_count,node_type,author_keynode,hashtag_keynode,extended_entities_count,verified
0,1192617057394708480,Armin Mudden,If that’s what the fires look like from up her...,2019-11-08T11:37:18.000000Z,"New South Wales, Australia",1,Positive,2.45,541.0,280,0,0,458,Root,False,False,1,False
1,1192684541913100288,Linda Roux,Progression of major fires today across NSW #N...,2019-11-08T16:05:28.000000Z,"New South Wales, Australia",1,Neutral,2.97,12942.0,609,0,0,673,Root,False,True,1,True
2,1192725213294804992,Carolyn Butler,#Smoke from #nswfires in the northeast is not ...,2019-11-08T18:47:05.000000Z,"New South Wales, Australia",8,Positive,2.34,148708.5,118,0,0,158,Root,False,True,2,True
3,1192766676854435840,NSW RFS,This shows the dangerous conditions that have ...,2019-11-08T21:31:50.000000Z,"New South Wales, Australia",1,Negative,3.64,297835.5,2879,0,0,5002,Root,True,True,1,True
4,1192932983407202304,Stephen Gösta Marquez-Lindkvist,Incredible vision from @NSWRFS of a crew from ...,2019-11-09T08:32:41.000000Z,Australia,1,Neutral,2.83,19101.0,364,26,85,639,Root,False,True,1,True


In [10]:
from datetime import datetime
# Function to convert datetime format
def convert_date_format(date_string):
    dt = datetime.strptime(date_string, "%Y-%m-%dT%H:%M:%S.%fZ")
    return dt.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z"
    
df['created_at_dt'] = df['created_at_dt'].apply(convert_date_format)

In [11]:
df.head()

,_id,deidentname,text,created_at_dt,location,dominant_topic,sentiment,influence_tweet_factor,influence_user,retweet_count,reply_count,quote_count,favourite_count,node_type,author_keynode,hashtag_keynode,extended_entities_count,verified
0,1192617057394708480,Armin Mudden,If that’s what the fires look like from up her...,2019-11-08T11:37:18.000Z,"New South Wales, Australia",1,Positive,2.45,541.0,280,0,0,458,Root,False,False,1,False
1,1192684541913100288,Linda Roux,Progression of major fires today across NSW #N...,2019-11-08T16:05:28.000Z,"New South Wales, Australia",1,Neutral,2.97,12942.0,609,0,0,673,Root,False,True,1,True
2,1192725213294804992,Carolyn Butler,#Smoke from #nswfires in the northeast is not ...,2019-11-08T18:47:05.000Z,"New South Wales, Australia",8,Positive,2.34,148708.5,118,0,0,158,Root,False,True,2,True
3,1192766676854435840,NSW RFS,This shows the dangerous conditions that have ...,2019-11-08T21:31:50.000Z,"New South Wales, Australia",1,Negative,3.64,297835.5,2879,0,0,5002,Root,True,True,1,True
4,1192932983407202304,Stephen Gösta Marquez-Lindkvist,Incredible vision from @NSWRFS of a crew from ...,2019-11-09T08:32:41.000Z,Australia,1,Neutral,2.83,19101.0,364,26,85,639,Root,False,True,1,True


In [12]:
df.to_csv('../../data/processed/Search_data.csv')

In [15]:
df["node_type"].unique()

array(['Root'], dtype=object)